## Recommendation using TF-IDF and Cosine Similarity

In [14]:
import os
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


DATASET_DIR = "dataset"
MODEL_DIR = "model_assets"

### 1. Load the clean data

In [15]:
try:
    df_recommender = pd.read_csv(os.path.join(DATASET_DIR, "books_cleaned.csv"))
    print("Cleaned Dataset loaded successfully.")

    try:
        df_full_metadata = pd.read_csv(os.path.join(DATASET_DIR, "books.csv"))
        print("Full Metadata Dataset loaded successfully.")
    except FileNotFoundError:
        print("Error: Full metadata dataset file not found.")
        df_full_metadata = None        

except FileNotFoundError:
    print("Error: Dataset file not found.")

Cleaned Dataset loaded successfully.
Full Metadata Dataset loaded successfully.


### 2. Calculate TF-IDF 

In [16]:
tfidf = TfidfVectorizer(
    stop_words='english',
    max_features=10000,
    min_df=5, # words must appear in at least 5 books
    max_df=0.85 # ignore words that appear in more than 85% of the books
)

print("Starting TF-IDF vectorization...")
tfidf_matrix = tfidf.fit_transform(df_recommender['combined_features'])

print("TF-IDF matrix shape:", tfidf_matrix.shape)


Starting TF-IDF vectorization...
TF-IDF matrix shape: (6810, 8419)


### 3. Similarity Calculation

In [17]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

print("Cosine similarity matrix shape:", cosine_sim.shape)

output_similarity_path = os.path.join(MODEL_DIR, "cosine_sim.npy")
np.save(output_similarity_path, cosine_sim)
print(f"Cosine similarity matrix saved to {output_similarity_path}")




Cosine similarity matrix shape: (6810, 6810)
Cosine similarity matrix saved to model_assets/cosine_sim.npy


### 4. Recommendation

In [18]:
def get_recommendations(title, cosine_sim, df_lean, df_full, N=10, print_output=True):

    indices = pd.Series(df_lean.index, index=df_lean['title']).drop_duplicates()

    # Check if the given title exists in the dataset
    if title not in indices:
        print(f"Error: Book title '{title}' not found.")
        return pd.DataFrame()

    # Get the index of the selected book
    idx = indices[title]

    sim_scores = sorted(
        enumerate(cosine_sim[idx]),
        key=lambda x: x[1],
        reverse=True
    )[1:N+1]


    # Separate book indices and similarity values
    book_indices, similarity_scores = zip(*sim_scores)

    recs = df_lean.iloc[list(book_indices)][['isbn13']].copy()
    recs['Similarity Score'] = similarity_scores


    display_cols = [
        'title', 'subtitle', 'authors',
        'categories', 'published_year', 'average_rating'
    ]

    # Remove duplicate ISBNs to ensure safe merging
    df_full_unique = df_full.drop_duplicates('isbn13')

    final_df = recs.merge(
        df_full_unique[['isbn13'] + display_cols],
        on='isbn13',
        how='left'
    ).fillna({
        'subtitle': '',
        'authors': '',
        'categories': ''
    })

    if print_output:
        print(f"\nTop {len(final_df)} Recommendations for: **{title}**")
        print("-" * 50)

        for i, row in final_df.iterrows():
            print(f"{i+1}. **{row['title']}**")

            # Print subtitle only if available
            if row['subtitle']:
                print(f"    ({row['subtitle']})")

            # Clean category formatting and show authors
            print(f"    {row['categories'].strip('[]\"\'')}, by {row['authors']}")

            # Show rating only if present
            if not pd.isna(row['average_rating']):
                print(f"    ⭐ {row['average_rating']:.2f}")

            # Show similarity score
            print(f"    (Similarity: {row['Similarity Score']:.4f})\n")

    # Return final recommendations as a clean DataFrame
    return final_df.reset_index(drop=True)


In [ ]:
# usage
recommedation_df = get_recommendations("The One Tree", cosine_sim, df_recommender, df_full_metadata, N=10, print_output=True)


Top 10 Recommendations for: **The One Tree**
--------------------------------------------------
1. **A Dark and Hungry God Arises**
    (The Gap Into Power)
    Fiction, by Stephen R. Donaldson
    ⭐ 4.08
    (Similarity: 0.4668)

2. **Reave the Just and Other Tales**
    Fiction, by Stephen R. Donaldson
    ⭐ 3.90
    (Similarity: 0.4494)

3. **Forbidden Knowledge**
    (The Gap Into Vision)
    Fiction, by Stephen R. Donaldson
    ⭐ 3.99
    (Similarity: 0.3874)

4. **Crime Novels**
    (American Noir of the 1930s and 40s)
    American fiction, by 
    ⭐ 4.36
    (Similarity: 0.3168)

5. **The Bachman Books**
    (Four Early Novels)
    Fiction, by Stephen King
    ⭐ 4.11
    (Similarity: 0.2563)

6. **The Gap Into Madness**
    (Chaos and Order)
    Hyland, Morn (Fictitious character), by Stephen R. Donaldson
    ⭐ 4.15
    (Similarity: 0.2478)

7. **Assassin's Apprentice**
    American fiction, by Robin Hobb
    ⭐ 4.15
    (Similarity: 0.2407)

8. **Best Science Fiction Stories of